In [ ]:
import pandas as pd

In [ ]:
csv_path_happiness = "Resources/2017_happiness.csv"
csv_path_libraries = "Resources/lmw_data_09_Feb_2021.csv"

happiness_df = pd.read_csv(csv_path_happiness)
libraries_df = pd.read_csv(csv_path_libraries, encoding='latin-1')

In [ ]:
happiness_df.head()

In [ ]:
libraries_df = libraries_df.loc[(libraries_df["Data Year"] == "2017"), :]
libraries_df = libraries_df.dropna()

#libraries_df.drop(libraries_df[libraries_df["Metric Data"]=="NaN"].index, inplace=True)

#library_df = libraries_df.drop(libraries_df[libraries_df["Metric Data"] == "NaN"].index, inplace=False)

#test_df = libraries_df.loc[libraries_df["Metric Data"]=="NaN", :]
libraries_df

#test_df

In [ ]:
combined_df = pd.merge(happiness_df, libraries_df, how='left', left_on=["Country"], right_on=["Country or Area"])
combined_df

In [ ]:
combined_df = combined_df.dropna()
combined_df.head()

In [ ]:
library_visits_df = libraries_df.loc[libraries_df["Metric"]=="Visits", :]

#library_visits_df.dropna()
library_visits_df["Metric Data (INT)"] = library_visits_df["Metric Data"].astype(int)


#library_visits_df.head(100)
type_group_df = library_visits_df.groupby(["Library Type"]).sum()

type_group_df

In [ ]:
# PAM CODE HERE - start

In [ ]:
# PAM CODE HERE - end

In [ ]:
# ZOE CODE HERE - start

In [ ]:
## **********Move up to cell 1 once team can coordinate***********
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## Overall happiness index 2017
# Get top 20
happiest_countries = happiness_df.loc[happiness_df["Happiness.Rank"] <= 20]

# set x axis and tick locations
countries = happiest_countries["Country"]
happiness = happiest_countries["Happiness.Score"]

# Create chart
plt.figure(figsize=(15,6))
happy_bar = plt.bar(countries, happiness, align="center")
plt.xticks(rotation="vertical")
plt.title("20 happiest countries by rank, 2017")

# Add happiness score text to bars
# for column in happy_bar:
#     height = column.get_height()
#     plt.text(column.get_x() + column.get_width()/2.0, height, '%d' % float(height), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
## Happiness index for countries which we also have library data for
# Get top 20
happiest_library_countries = combined_df.loc[combined_df["Happiness.Score"] > 6.1]

# set x axis and tick locations
countries = happiest_library_countries["Country"]
happiness = happiest_library_countries["Happiness.Score"]

# Create chart
plt.figure(figsize=(15,6))
plt.bar(countries, happiness, align="center")
plt.xticks(rotation="vertical")
plt.title("20 happiest countries with library usage data by rank, 2017")

plt.tight_layout()
plt.show()

In [ ]:
# ZOE CODE HERE - end

In [ ]:
# MALLORY CODE HERE - start

In [ ]:
# MALLORY CODE HERE - end

In [ ]:
# ELLIE CODE HERE - start

In [ ]:
# ELLIE CODE HERE - end